In [ ]:
import importlib, traceback, pplx_client, inspect
print("module file:", pplx_client.__file__)
print("has PPLXClient?", "PPLXClient" in dir(pplx_client))
if "PPLXClient" not in dir(pplx_client):
    # 파일 내용을 실제로 확인 (앞 60줄)
    import pathlib
    src = pathlib.Path(pplx_client.__file__).read_text(encoding="utf-8")
    print("--- file head ---")
    print("\n".join(src.splitlines()[:60]))


In [ ]:
import pandas as pd

In [26]:
# 데이터 불러오기

df_items = pd.read_csv('data/jewelry_items_word_with_category.csv')
df_sentiment = pd.read_csv('data/jewelry_sentiment_words.csv')
df_clean = pd.read_csv('data/cafe_posts_clean.csv')


In [22]:
df_items.head()

,brand_id,brand_name,line_id,line_name,category_id,category_name,class_type,relative_word
0,b001,IWC,NaN,NaN,NaN,NaN,NaN,"IWC,IW,International Watch Co"
1,b001,IWC,l117,파일럿 워치,NaN,NaN,NaN,"파일럿 워치,파일럿워치"
2,b001,IWC,l123,포르투기저,NaN,NaN,NaN,"포르투기저,포르투,Portugieser"
3,b001,IWC,l117,파일럿 워치,c006,시계,플래그십,"IWC 시계, IWC시계, IWC 워치, IWC워치, IWC 손목시계, IWC손목시..."
4,b001,IWC,l123,포르투기저,c006,시계,플래그십,"IWC 시계, IWC시계, IWC 워치, IWC워치, IWC 손목시계, IWC손목시..."


In [17]:
df_sentiment.head()

,category,words
0,디자인_긍정,"예쁘다, 이쁘다, 이뻐, 예뻐, 이쁜, 예쁜, 아름답다, 아름다운, 멋있다, 멋진,..."
1,가격_긍정,"가성비, 혜자, 갓성비, 가성비좋다, 가성비갓, 가성비킹, 혜택, 저렴하다, 저렴한..."
2,만족도_긍정,"만족, 만족해, 만족스럽다, 만족스러운, 좋다, 좋아, 좋은, 좋네, 마음에들다, ..."
3,추천_긍정,"추천, 강추, 강력추천, 완전추천, 무조건추천, 적극추천, 인생템, 인정, 인정해,..."
4,디자인_부정,"못생겼다, 못생긴, 이상하다, 이상한, 어색하다, 어색한, 이상해, 별로다, 별로네..."


In [27]:
df_clean.head()

,naver_article_id,writer_nickname,released_at,view_count,title,like_count,comment_count,comments,text
0,5536787,2024Y,2025-08-19 23:38:00,345,불가리 뱀이냐 까르띠에 팬더냐 🤣 뭐가 더 이쁜가요?,0,4,앗 전 압도적으로 2번이요|불가리 착용해보심 실물 넘예뻐요!!\n링부분도 투보가스라...,​\n​\n데일리나 레이어드는 팬서가 더 잘 될 것 같는데\n뱀은 빡 화려하니 롱런...
1,5536774,가오리맘,2025-08-19 23:22:00,877,2천만원 예산 골라주세요~,0,10,11111111|1번이 뭔가 잘하고 다닐 것 같긴해요ㅎㅎ|1번요|1번이군요 ㅎㅎ|화...,​\n현재 가지고 있는 제품은 모두 로즈골드구요\n-반클리프 빼를리 팔찌 xs\n-...
2,5536732,아미노쿨,2025-08-19 22:37:00,2286,20대 동기부여 용으로 까르띠에 어떨까요?,0,21,남자분이 차기엔 너무 작을거같아요 스몰이상은\n가셔야.. 사진모델은 미니에요|공홈 ...,20대 초반 대학생인데 조금 무리해서 탱크 루이 사고싶은데 시크님들은 어떤 의견인지...
3,5536722,리안s,2025-08-19 22:28:00,543,팬더/탱크 보려면 주말은 불가할까요ㅠㅠ,1,14,삭제된 댓글입니다.|헉 그렇군여 넘 멋모르고 돌아다닐뻔 했네요 감사합니다!!|솔로가...,팬더/탱크머스트/ 탱크루이 시착해보고\n어울리는 컬러랑 디자인 고르고 싶은데\n주말...
4,5536667,졸업시즌,2025-08-19 21:51:00,67,"프레드 샹스 미듐 화이트,골드 골라주세요",0,1,옐로우 골드요,​\n은장 사고 싶었는데 더 나이들면 금장도 이쁠것같아서 고민됩니다 ㅠ 현재 40초...


In [ ]:
"""
pplx_client.py
Perplexity API 호출용 기본 클라이언트 (OpenAI SDK 호환)
"""

import os
import pandas as pd
from dotenv import load_dotenv
from openai import OpenAI


class PPLXClient:
    """Perplexity API 래퍼 클래스"""

    def __init__(self, model: str = "sonar"):
        load_dotenv()
        api_key = os.getenv("PPLX_API_KEY")
        if not api_key:
            raise ValueError("환경변수 PPLX_API_KEY를 설정해주세요.")

        self.client = OpenAI(api_key=api_key, base_url="https://api.perplexity.ai")
        self.model = model

    def ask(self, prompt: str, system_prompt: str = "Be precise and concise.") -> str:
        """단건 호출"""
        resp = self.client.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt},
            ],
        )
        return resp.choices[0].message.content

    def ask_stream(self, prompt: str, system_prompt: str = "Be precise and concise."):
        """스트리밍 호출"""
        stream = self.client.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt},
            ],
            stream=True,
        )
        for chunk in stream:
            delta = chunk.choices[0].delta
            if delta and delta.content:
                yield delta.content

    @staticmethod
    def preview_table(df: pd.DataFrame, n: int = 5) -> str:
        """데이터 미리보기 텍스트 생성"""
        return df.head(n).to_markdown(index=False)

    def ask_about_csv(self, df: pd.DataFrame, question: str) -> str:
        """CSV 미리보기 + 질문"""
        context = self.preview_table(df, n=5)
        prompt = (
            "당신은 초보 데이터 분석가입니다. 아래 표를 참고해 질문에 답하세요.\n\n"
            f"{context}\n\n질문: {question}"
        )
        return self.ask(prompt)


In [ ]:
%pip install -U openai python-dotenv pandas

In [ ]:
%pip install tabulate

In [ ]:
# CSV 로드
df_items = pd.read_csv('data/jewelry_items_word_with_category.csv')

from pplx_client import PPLXClient
client = PPLXClient(model="sonar")

print(client.ask("한 문장으로 자기소개 예시를 만들어줘.", temperature=0.2))
print(client.ask_about_df(df_items, "이 데이터의 주요 컬럼 3개만 뽑아 설명해줘."))
for tok in client.ask_stream("한국의 수도는?", temperature=0.0):
    print(tok, end="")


In [ ]:
import pandas as pd
from pplx_client import PPLXClient

# 1) CSV 불러오기
df_items = pd.read_csv('data/jewelry_items_word_with_category.csv')
df_sentiment = pd.read_csv('data/jewelry_sentiment_words.csv')


# 2) 클라이언트 초기화
pplx = PPLXClient(model="sonar")

# 3) 일반 질문
print(pplx.ask("한 문장으로 자기소개 예시를 만들어줘."))

# 4) CSV 기반 질문
print(pplx.ask_about_df(df_items, "이 데이터의 주요 컬럼 3개만 뽑아 설명해줘."))

# 5) 스트리밍 출력
for token in pplx.ask_stream("스트리밍으로 한국의 수도를 알려줘."):
    print(token, end="", flush=True)


In [ ]:
import pandas as pd
from pplx_client import PPLXClient

# CSV 로드
df_items = pd.read_csv('data/jewelry_items_word_with_category.csv')
df_sentiment = pd.read_csv('data/jewelry_sentiment_words.csv')

# 클라이언트
client = PPLXClient(model="sonar")  # 필요 시 "sonar-pro" 등

# 1) 단건 질문
print(client.ask("데이터 분석가의 역량 알려줘", temperature=0.7))

# 2) 메시지 배열
msgs = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Summarize the benefits of model quantization in 2 bullets."},
]
print(client.chat(msgs, temperature=0.3))

# 3) 스트리밍
for token in client.ask_stream("한국의 수도는?", temperature=0.0):
    print(token, end="", flush=True)

# 4) DataFrame 기반 Q&A
print(client.ask_about_df(df_items, "이 데이터의 주요 컬럼 3개만 뽑아 설명해줘."))


In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()  # .env 읽기
print("키 확인:", os.getenv("PPLX_API_KEY")[:8] if os.getenv("PPLX_API_KEY") else None)


In [ ]:
import os
print(os.getcwd())

In [ ]:
from dotenv import load_dotenv
load_dotenv(dotenv_path="/Users/t2024-m0246/Downloads/sparta-project/luxury/.env")


In [ ]:
import dotenv
print("dotenv 모듈 위치:", dotenv.__file__)
print("dotenv 모듈 버전:", dotenv.__version__)

In [ ]:
%pip show python-dotenv

In [ ]:
# 잘못된 dotenv 제거

%pip uninstall -y dotenv

In [ ]:
# 올바른 패키지 설치/업데이트

%pip install -U python-dotenv

In [ ]:
import importlib.metadata
print(importlib.metadata.version("python-dotenv"))

In [ ]:
pip install --upgrade pip

In [ ]:
%pip install -U python-dotenv

In [ ]:
import importlib.metadata
print(importlib.metadata.version("python-dotenv"))

세팅완료 !!!

In [ ]:
import pandas as pd
from pplx_client import PPLXClient

# 1) CSV 불러오기
df_items = pd.read_csv('data/jewelry_items_word_with_category.csv')
df_sentiment = pd.read_csv('data/jewelry_sentiment_words.csv')
df_clean = pd.read_csv('data/cafe_posts_clean.csv')

# 2) 클라이언트 초기화
pplx = PPLXClient(model="sonar")

# 3) 일반 질문
# print(pplx.ask("한 문장으로 자기소개 예시를 만들어줘."))

# 4) CSV 기반 질문
print(pplx.ask_about_df(df_items, df_sentiment, df_clean "긍정 "))

# 5) 스트리밍 출력
# for token in pplx.ask_stream("스트리밍으로 한국의 수도를 알려줘."):
#     print(token, end="", flush=True)

SyntaxError: invalid syntax (944047395.py, line 16)

In [ ]:
df_items, df_sentiment, df_clean 를 함께 활용해서 → 긍정 단어가 가장 많이 나온 제품명을 찾기

In [19]:
df_items.head()

,brand_id,brand_name,line_id,line_name,category_id,category_name,class_type,relative_word
0,b001,IWC,NaN,NaN,NaN,NaN,NaN,"IWC,IW,International Watch Co"
1,b001,IWC,l117,파일럿 워치,NaN,NaN,NaN,"파일럿 워치,파일럿워치"
2,b001,IWC,l123,포르투기저,NaN,NaN,NaN,"포르투기저,포르투,Portugieser"
3,b001,IWC,l117,파일럿 워치,c006,시계,플래그십,"IWC 시계, IWC시계, IWC 워치, IWC워치, IWC 손목시계, IWC손목시..."
4,b001,IWC,l123,포르투기저,c006,시계,플래그십,"IWC 시계, IWC시계, IWC 워치, IWC워치, IWC 손목시계, IWC손목시..."


In [20]:
df_sentiment.head()

,category,words
0,디자인_긍정,"예쁘다, 이쁘다, 이뻐, 예뻐, 이쁜, 예쁜, 아름답다, 아름다운, 멋있다, 멋진,..."
1,가격_긍정,"가성비, 혜자, 갓성비, 가성비좋다, 가성비갓, 가성비킹, 혜택, 저렴하다, 저렴한..."
2,만족도_긍정,"만족, 만족해, 만족스럽다, 만족스러운, 좋다, 좋아, 좋은, 좋네, 마음에들다, ..."
3,추천_긍정,"추천, 강추, 강력추천, 완전추천, 무조건추천, 적극추천, 인생템, 인정, 인정해,..."
4,디자인_부정,"못생겼다, 못생긴, 이상하다, 이상한, 어색하다, 어색한, 이상해, 별로다, 별로네..."


In [30]:
df_clean.head()

,naver_article_id,writer_nickname,released_at,view_count,title,like_count,comment_count,comments,text
0,5536787,2024Y,2025-08-19 23:38:00,345,불가리 뱀이냐 까르띠에 팬더냐 🤣 뭐가 더 이쁜가요?,0,4,앗 전 압도적으로 2번이요|불가리 착용해보심 실물 넘예뻐요!!\n링부분도 투보가스라...,​\n​\n데일리나 레이어드는 팬서가 더 잘 될 것 같는데\n뱀은 빡 화려하니 롱런...
1,5536774,가오리맘,2025-08-19 23:22:00,877,2천만원 예산 골라주세요~,0,10,11111111|1번이 뭔가 잘하고 다닐 것 같긴해요ㅎㅎ|1번요|1번이군요 ㅎㅎ|화...,​\n현재 가지고 있는 제품은 모두 로즈골드구요\n-반클리프 빼를리 팔찌 xs\n-...
2,5536732,아미노쿨,2025-08-19 22:37:00,2286,20대 동기부여 용으로 까르띠에 어떨까요?,0,21,남자분이 차기엔 너무 작을거같아요 스몰이상은\n가셔야.. 사진모델은 미니에요|공홈 ...,20대 초반 대학생인데 조금 무리해서 탱크 루이 사고싶은데 시크님들은 어떤 의견인지...
3,5536722,리안s,2025-08-19 22:28:00,543,팬더/탱크 보려면 주말은 불가할까요ㅠㅠ,1,14,삭제된 댓글입니다.|헉 그렇군여 넘 멋모르고 돌아다닐뻔 했네요 감사합니다!!|솔로가...,팬더/탱크머스트/ 탱크루이 시착해보고\n어울리는 컬러랑 디자인 고르고 싶은데\n주말...
4,5536667,졸업시즌,2025-08-19 21:51:00,67,"프레드 샹스 미듐 화이트,골드 골라주세요",0,1,옐로우 골드요,​\n은장 사고 싶었는데 더 나이들면 금장도 이쁠것같아서 고민됩니다 ㅠ 현재 40초...


In [33]:
import pandas as pd
import numpy as np
import re

# ---------- 0) 유틸 ----------
def split_csv_list(s: str) -> list[str]:
    """쉼표로 구분된 문자열을 리스트로. 공백/빈값/NaN 안전 처리."""
    if pd.isna(s):
        return []
    return [x.strip() for x in str(s).split(",") if x.strip()]

def safe_join_name(brand, line):
    """브랜드/라인 조합으로 제품명 라벨 생성"""
    b = str(brand).strip() if pd.notna(brand) else ""
    l = str(line).strip() if pd.notna(line) else ""
    return f"{b} {l}".strip() if l else b

# ---------- 1) 긍정 단어 사전 구축 ----------
# df_sentiment: [category, words]
mask_pos = df_sentiment["category"].astype(str).str.endswith("긍정")
pos_words = []
for words in df_sentiment.loc[mask_pos, "words"].dropna():
    pos_words.extend(split_csv_list(words))

# 중복 제거 & 길이 내림차순(긴 단어를 먼저 매칭해 부분중복 영향↓)
pos_words = sorted(set(pos_words), key=len, reverse=True)
# 긍정 단어 정규식 (특수문자 이스케이프)
pos_re = re.compile("|".join(map(re.escape, pos_words))) if pos_words else None

# ---------- 2) 제품(별칭) 테이블 준비 ----------
# df_items: [brand_id, brand_name, line_id, line_name, category_id, category_name, class_type, relative_word]
items = df_items.copy()

# 제품 라벨(출력용)
items["product_label"] = items.apply(lambda r: safe_join_name(r.get("brand_name"), r.get("line_name")), axis=1)

# 별칭 리스트: relative_word + brand_name + line_name
def build_aliases(row) -> list[str]:
    aliases = set(split_csv_list(row.get("relative_word")))
    if pd.notna(row.get("brand_name")) and str(row.get("brand_name")).strip():
        aliases.add(str(row.get("brand_name")).strip())
    if pd.notna(row.get("line_name")) and str(row.get("line_name")).strip():
        aliases.add(str(row.get("line_name")).strip())
    # 길이 내림차순 (긴 별칭 우선 매칭)
    return sorted({a for a in aliases if a}, key=len, reverse=True)

items["aliases"] = items.apply(build_aliases, axis=1)

# 별칭 정규식(제품별)
def compile_alias_re(aliases):
    if not aliases:
        return None
    return re.compile("|".join(map(re.escape, aliases)))
items["alias_re"] = items["aliases"].apply(compile_alias_re)

# ---------- 3) 콘텐츠 합치기 ----------
# df_clean: [title, text, comments, ...]
clean = df_clean.copy()
for c in ["title", "text", "comments"]:
    if c not in clean.columns:
        clean[c] = ""
clean["content"] = (
    clean["title"].fillna("").astype(str) + "\n" +
    clean["text"].fillna("").astype(str) + "\n" +
    clean["comments"].fillna("").astype(str)
)

contents = clean["content"].tolist()  # 속도 때문에 리스트로 빼두기

# ---------- 4) 제품별 긍정 단어 카운트 ----------
results = []
for idx, row in items.iterrows():
    alias_re = row["alias_re"]
    if alias_re is None or pos_re is None:
        pos_count = 0
        doc_hits = 0
    else:
        pos_count = 0
        doc_hits = 0
        for doc in contents:
            if alias_re.search(doc):              # 제품 언급된 글만 집계
                doc_hits += 1
                pos_count += len(pos_re.findall(doc))
    results.append({
        "brand_id": row.get("brand_id"),
        "product_label": row["product_label"],
        "positive_word_count": int(pos_count),
        "mentioned_docs": int(doc_hits),
        "aliases": row["aliases"],
        "category_name": row.get("category_name"),
        "line_name": row.get("line_name"),
    })

result_df = pd.DataFrame(results)

# ---------- 5) 상위 결과 확인 ----------
topN = (
    result_df.sort_values(["positive_word_count", "mentioned_docs"], ascending=[False, False])
    .head(10)
    .reset_index(drop=True)
)

print("✅ 긍정 단어가 많이 언급된 상위 제품")
print(topN[["product_label", "positive_word_count", "mentioned_docs"]].to_string(index=False))


✅ 긍정 단어가 많이 언급된 상위 제품
product_label  positive_word_count  mentioned_docs
        판도라 미                45074            5052
        판도라 미                45074            5052
        판도라 미                45074            5052
        판도라 미                45074            5052
         디올 위                35363            3191
         디올 위                35363            3191
     샤넬 보이프렌드                33817            3162
       구찌 다이브                33379            3318
      까르띠에 러브                30375            4157
      까르띠에 러브                30375            4157
